<a href="https://colab.research.google.com/github/antonsysoev/DeepLearning/blob/main/Basic_Examples_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

Переменная в TensorFlow — это некий буфер в памяти, который содержит тензоры.
Переменные нужно явным образом инициализировать. Чтобы объявить переменную,
нужно задать способ ее инициализации; при желании можно также назначить
ей имя, на которое можно будет потом ссылаться. Например:

In [ ]:
w = tf.Variable(tf.random.normal([3, 2], mean=0.0, stddev=0.4), name='weights')
b = tf.Variable(tf.zeros([2]), name='biases')

В этом коде мы объявили две переменные: w с именем weights и b с именем biases.
Для переменных можно явным образом указать, где именно им нужно находиться
в памяти; например, если вы хотите объявить переменную на первой (нулевой)
видеокарте, это можно сделать так:

In [ ]:
with tf.device('/gpu:0'):
  w = tf.Variable(tf.random.normal([3, 2], mean=0.0, stddev=0.4), name='weights')

В TensorFlow реализован полный набор операций над тензорами из NumPy
с поддержкой матричных вычислений над массивами разной формы и конвертирования
между этими формами (broadcasting). Например, в реальных задачах часто
возникает необходимость к каждому столбцу матрицы поэлементно добавить один
и тот же вектор. В TensorFlow это делается самым простым из возможных способов:

In [ ]:
m = tf.Variable(tf.random.normal([10, 100], mean=0.0, stddev=0.4), name='matrix')
v = tf.Variable(tf.random.normal([100], mean=0.0, stddev=0.4), name='vector')
result = m + v

In [ ]:
def linear_transform(vec,shape):
  w = tf.Variable(tf.random.normal(shape, mean=0.0, stddev=1.0), name='matrix')
  return tf.matmul(vec,w)

In [ ]:
import numpy as np, tensorflow as tf
tf.compat.v1.disable_eager_execution()
n_samples, batch_size, num_steps = 1000,100,30000

# y = 2x + 1

X_data = np.random.uniform(1,10,(n_samples,1))
y_data = 2 * X_data + 1 + np.random.normal(0,2,(n_samples,1))

X = tf.compat.v1.placeholder(tf.float32,shape=(batch_size,1))
y = tf.compat.v1.placeholder(tf.float32,shape=(batch_size,1))

with tf.compat.v1.variable_scope('linear-regression'):
  k = tf.Variable(tf.random.normal((1,1)), name = 'slope')
  b = tf.Variable(tf.zeros((1,)), name = 'bias')

y_pred = tf.matmul(X,k) + b
loss = tf.reduce_sum((y-y_pred)**2)
optimizer = tf.compat.v1.train.GradientDescentOptimizer(0.00001).minimize(loss)

display_step = 100
with tf.compat.v1.Session() as sess:
  sess.run(tf.compat.v1.global_variables_initializer())
  for i in range(num_steps):
    indices = np.random.choice(n_samples,batch_size)
    X_batch, y_batch = X_data[indices], y_data[indices]
    _, loss_val, k_val, b_val = sess.run([optimizer,loss,k,b],
                                         feed_dict = {X:X_batch, y:y_batch})
    if (i+1) % display_step == 0:
      print('Эпоха %d: %.8f, k = %.4f, b = %.4f' %
            (i+1, loss_val, k_val, b_val))


Эпоха 100: 409.14382935, k = 2.0650, b = 0.5001
Эпоха 200: 374.13916016, k = 2.0590, b = 0.5167
Эпоха 300: 330.15701294, k = 2.0606, b = 0.5329
Эпоха 400: 309.19104004, k = 2.0577, b = 0.5470
Эпоха 500: 489.74005127, k = 2.0558, b = 0.5623
Эпоха 600: 379.61108398, k = 2.0544, b = 0.5779
Эпоха 700: 369.54577637, k = 2.0572, b = 0.5921
Эпоха 800: 371.01528931, k = 2.0597, b = 0.6016
Эпоха 900: 394.51525879, k = 2.0503, b = 0.6144
Эпоха 1000: 312.49621582, k = 2.0451, b = 0.6284
Эпоха 1100: 339.07272339, k = 2.0439, b = 0.6402
Эпоха 1200: 386.43487549, k = 2.0389, b = 0.6469
Эпоха 1300: 390.90298462, k = 2.0434, b = 0.6574
Эпоха 1400: 368.90887451, k = 2.0490, b = 0.6701
Эпоха 1500: 397.70520020, k = 2.0335, b = 0.6806
Эпоха 1600: 238.56654358, k = 2.0419, b = 0.6914
Эпоха 1700: 329.07550049, k = 2.0271, b = 0.6987
Эпоха 1800: 428.91787720, k = 2.0381, b = 0.7097
Эпоха 1900: 385.42565918, k = 2.0376, b = 0.7194
Эпоха 2000: 351.43365479, k = 2.0394, b = 0.7277
Эпоха 2100: 377.11306763, k =

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Input, Dense, Activation

In [ ]:
logr = Sequential()
logr.add(Dense(1,input_dim = 2, activation = 'sigmoid'))
logr.compile(loss = 'binary_crossentropy', optimizer='sgd', metrics = ['accuracy'])

In [ ]:
def sampler(n,x,y):
  return np.random.normal(size=[n,2])+[x,y]

In [ ]:
def sample_data(n=1000, p0=(-1,-1), p1=(1,1)):
  zeros = np.zeros((n,1))
  ones = np.ones((n,1))
  labels = np.vstack([zeros,ones])
  z_sample = sampler(n,x=p0[0], y=p0[1])
  o_sample = sampler(n,x=p1[0], y=p1[1])
  return np.vstack([z_sample,o_sample]), labels

In [ ]:
X_train, Y_train = sample_data()
X_test, Y_test = sample_data(100)

In [ ]:
logr.fit(X_train, Y_train, batch_size=16, nb_epoch=100, verbose=1, validation_data=(X_test,Y_test))

Train on 2000 samples, validate on 200 samples
Epoch 1/100
2000/2000 [==============================] - 0s 168us/sample - loss: 0.3133 - accuracy: 0.9080 - val_loss: 0.3212 - val_accuracy: 0.8750
Epoch 2/100
2000/2000 [==============================] - 0s 53us/sample - loss: 0.2768 - accuracy: 0.9190 - val_loss: 0.2953 - val_accuracy: 0.8750
Epoch 3/100
  16/2000 [..............................] - ETA: 0s - loss: 0.1549 - accuracy: 1.0000

/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


2000/2000 [==============================] - 0s 64us/sample - loss: 0.2561 - accuracy: 0.9215 - val_loss: 0.2792 - val_accuracy: 0.8750
Epoch 4/100
2000/2000 [==============================] - 0s 63us/sample - loss: 0.2429 - accuracy: 0.9225 - val_loss: 0.2682 - val_accuracy: 0.8700
Epoch 5/100
2000/2000 [==============================] - 0s 56us/sample - loss: 0.2338 - accuracy: 0.9245 - val_loss: 0.2603 - val_accuracy: 0.8700
Epoch 6/100
2000/2000 [==============================] - 0s 65us/sample - loss: 0.2272 - accuracy: 0.9250 - val_loss: 0.2544 - val_accuracy: 0.8700
Epoch 7/100
2000/2000 [==============================] - 0s 62us/sample - loss: 0.2222 - accuracy: 0.9240 - val_loss: 0.2498 - val_accuracy: 0.8750
Epoch 8/100
2000/2000 [==============================] - 0s 66us/sample - loss: 0.2183 - accuracy: 0.9240 - val_loss: 0.2461 - val_accuracy: 0.8800
Epoch 9/100
2000/2000 [==============================] - 0s 63us/sample - loss: 0.2152 - accuracy: 0.9250 - val_loss: 0.2432